Takes the original scraped data and migrates it to an SQL Server database.

In [120]:
import pandas as pd
import numpy as np
from datetime import datetime
import bidict
import pyarrow.parquet as pq
from sqlalchemy import create_engine, VARCHAR, Integer
import pyodbc
import pickle

In [22]:
conn = pyodbc.connect('DRIVER={SQL Server};Server=.;Database=steam_data;Trusted_Connection=True;')

query = "SELECT * FROM test"
cursor = conn.cursor()
cursor.execute(query)
rows = cursor.fetchall()
for row in rows :
    print(row)

(1, 'tom')
(2, 'ted')
(3, 'tub')


In [24]:
with open('../data/interim/recently_played_cleaned.pkl', 'rb+') as file:
    rp_df = pickle.load(file)

In [87]:
connection_string = "mssql+pyodbc://./steam_data?trusted_connection=yes&driver=ODBC+Driver_17+for+SQL+Server"
engine = create_engine(connection_string)

In [76]:
dtypes = {
    'player': VARCHAR(17),
    'app_id':Integer,
    'playtime_2w':Integer,
    'playtime_f':Integer
}

In [77]:
rp_df.to_sql('recently_played', con=engine, if_exists='replace', index=False, dtype=dtypes)

DBAPIError: (pyodbc.Error) ('HY104', '[HY104] [Microsoft][ODBC SQL Server Driver]Invalid precision value (0) (SQLBindParameter)')
[SQL: SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE ([INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max)) OR [INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max))) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_SCHEMA] = CAST(? AS NVARCHAR(max))]
[parameters: ('BASE TABLE', 'VIEW', 'recently_played', 'dbo')]
(Background on this error at: https://sqlalche.me/e/20/dbapi)

In [72]:
rp_df = rp_df.rename(columns={'user':'player'})

In [73]:
rp_df

,player,app_id,playtime_2w,playtime_f
0,76561198006340897,730,810,128229
1,76561198042662726,2497900,51,176
2,76561198342674289,359550,25,53668
3,76561198010015393,553850,519,519
4,76561197999713911,1059990,4,429
...,...,...,...,...
4181496,76561198120695201,271590,52,10703
4181497,76561198283301095,437920,369,369
4181498,76561198041431864,1205450,447,493
4181499,76561198954330253,570,2169,33812


In [38]:
len(rp_df['user'][0])

17

In [78]:
with open('../data/interim/1 - Games DF - Wrangled.pkl', 'rb+') as file:
    games_df = pickle.load(file)

In [79]:
games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100894 entries, 0 to 100893
Data columns (total 22 columns):
 #   Column                       Non-Null Count   Dtype   
---  ------                       --------------   -----   
 0   app_id                       100894 non-null  int64   
 1   title                        100894 non-null  object  
 2   developer                    100894 non-null  object  
 3   publisher                    100894 non-null  object  
 4   description                  100894 non-null  object  
 5   release_date                 100796 non-null  object  
 6   price                        82970 non-null   float64 
 7   price_category               82970 non-null   category
 8   number_of_reviews            35269 non-null   float64 
 9   positive_review_percent      35269 non-null   float64 
 10  relevant_langs_comments_sum  100894 non-null  int32   
 11  tags                         100894 non-null  object  
 12  tag_list                     100894 non-null

In [154]:
game_tags = games_df[['app_id', 'tags', 'tag_list', 'date_scraped']].rename(columns={'tags':'main_tags', 'tag_list':'other_tags'})

In [155]:
cols = ['main_tags', 'other_tags']

for col in cols :
    list_to_string(game_tags, col)

In [157]:
game_tags.to_sql('game_tags', con=engine, index=False, if_exists='replace')

286

In [126]:
game_spec = games_df[['app_id', 'title', 'developer', 'publisher', 'description', 'release_date', 'price', 'game_page_link', 'date_scraped']]

In [129]:
game_reviews = games_df[['app_id', 'number_of_reviews', 'positive_review_percent', 'relevant_langs_comments_sum', 'date_scraped']]

In [132]:
game_reviews.to_sql('game_reviews', con=engine, if_exists='replace', index=False)

334

In [134]:
game_langs = games_df[['app_id', 'interface_languages', 'full_audio_languages', 'subtitles_languages', 'date_scraped']]

In [138]:
cols = ['interface_languages', 'full_audio_languages', 'subtitles_languages']

for col in cols :
    list_to_string(game_langs, col)

C:\Users\Josh\AppData\Local\Temp\ipykernel_25444\1019706306.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(lambda x: str(x).strip("[]"))
C:\Users\Josh\AppData\Local\Temp\ipykernel_25444\1019706306.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(lambda x: x.strip("'"))
C:\Users\Josh\AppData\Local\Temp\ipykernel_25444\1019706306.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [140]:
game_langs.to_sql('game_langs', con=engine, index=False, if_exists='replace')

334

In [137]:
def list_to_string(df=None, col=None) :
    df[col] = df[col].apply(lambda x: str(x).strip("[]"))
    df[col] = df[col].apply(lambda x: x.strip("'"))
    df[col] = df[col].apply(lambda x: x.replace("', '", ", "))

In [89]:
connection_string = "mssql+pyodbc://./steam_data?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server"
engine = create_engine(connection_string)

In [128]:
game_spec.to_sql('games', con=engine, if_exists='replace', index=False)

5

In [146]:
review_table = None

In [145]:
review_df = pd.read_parquet('../data/interim/cleaned_reviews.parquet')

In [151]:
review_df.to_sql('reviews', con=engine, index=False, if_exists='replace')

54